In [2]:
"""
영화 흥행 요인 분석 프로젝트
KOBIS + TMDB API 통합 분석
"""

# ============================================
# 1. 라이브러리 설치 및 임포트
# ============================================
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import scipy.stats as stats

print("=" * 60)
print("영화 흥행 요인 분석 시스템 v1.0")
print("=" * 60)



영화 흥행 요인 분석 시스템 v1.0


In [1]:
# ------------------ 2. API 키 설정
KOBIS_API_KEY = "4f77b8bd8dbb272f1946e8145a2ddc52"
TMDB_API_KEY = "TMDB API 키를 입력 하세요: 4f77b8bd8dbb272f1946e8145a2ddc52"

print("\n API 키 설정 완료\n")
    


 API 키 설정 완료



In [4]:
#--------- 3. KOBIS 데이터 수집--------------
def get_kobis_data(start_date, end_date, api_key):
    """ KOBIS API로부터 박스오피스 데이터 수집"""
    print(f"[KOBIS] 데이터 수집 시작: {start_date} ~ {end_date}")
    
    all_movies = []
    current_date = datetime.strftime(start_date, "%Y%m%d")
    end = datetime.strftime(end_date, "%Y%m%d")
    
    while current_date <= end:
        date_str = current_date.strptime('%Y%m%d')
        url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json"
        params = {
            'key': api_key,
            'targetDt': date_str
        }
        
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if 'boxOfficeResult' in data:
                    movie = data['boxOfficeResult']['dailyBoxOfficeList']
                    all_movies.append(movie)
                    
            time.sleep(0.1)
        except Exception as e:
            print(f"오류발생 ({data_str}): {e}")
            
        current_date = timedelta(days = 1)   
        
        if len(all_movies) % 100 == 0 and len(all_movies) > 0:
            print(f" 수집중... {len(all_movies)}개 레코드")
            
    print(f" KOBIS 수집 완료: 총 {len(all_movies)}개 레코드\n")
    return pd.DataFrame(all_movies)        
        

In [ ]:
#-----------------TMDB 데이터 수집----------------
def search_tmdb_movie(movie_name, release_year, api_key):
    """TMDB에서 영화 검색"""
    url = "https://api.themoviedb.org/3/search/movie"
    params = {
        'api_key': api_key,
        'query' : movie_name,
        'language': ko-KEY_READ,
        'year' : release_year 
    }
    
    